In [ ]:
import numpy as np
import pandas as pd

# 读取训练数据和测试数据
df_train = pd.read_excel('train.xlsx')
df_test = pd.read_excel('test_A榜.xlsx')

# 打印数据的尺寸, 一共有49个特征
print(f"训练数据的数据大小为：{df_train.shape}")
print(f"测试A榜的数据大小为：{df_test.shape}")

def count_q_and_nan(df,feature,ratio=0.5):
    """
    ratio是？和nan数据加起来数量与总数量比例，控制特征的噪音不能太多
    越小越严格，剔除的特征越多
    """
    total_invalid_num = (df.shape[0]-sum(df[feature].value_counts()))+df[feature].value_counts()["?"]
    if total_invalid_num/df.shape[0] > ratio:
        print(f"Remove feature {feature}!")
        df.drop([feature],inplace=True,axis = 1)
        return feature

# CNT代表计数

feature_info = {
    'AGN_CNT_RCT_12_MON':'float',
    'ICO_CUR_MON_ACM_TRX_TM':'float',
    'NB_RCT_3_MON_LGN_TMS_AGV':'float',
    'AGN_CUR_YEAR_AMT':'float',
    'AGN_CUR_YEAR_WAG_AMT':'float',
    'AGN_AGR_LATEST_AGN_AMT':'float',
    'ICO_CUR_MON_ACM_TRX_AMT':'float',
    'COUNTER_CUR_YEAR_CNT_AMT':'float',
    'PUB_TO_PRV_TRX_AMT_CUR_YEAR':'float',
    'MON_12_EXT_SAM_TRSF_IN_AMT':'float',
    'MON_12_EXT_SAM_TRSF_OUT_AMT':'float',
    'MON_12_EXT_SAM_NM_TRSF_OUT_CNT':'float',
    'MON_12_EXT_SAM_AMT':'float',
    'CUR_MON_EXT_SAM_CUST_TRSF_IN_AMT':'float',
    'CUR_MON_EXT_SAM_CUST_TRSF_OUT_AMT':'float',
    'MON_12_CUST_CNT_PTY_ID':'str',
    'MON_12_TRX_AMT_MAX_AMT_PCTT':'float',
    'CUR_YEAR_MON_AGV_TRX_CNT':'float',
    'MON_12_AGV_TRX_CNT':'float',
    'MON_12_ACM_ENTR_ACT_CNT':'float',
    'MON_12_AGV_ENTR_ACT_CNT':'float',
    'MON_12_ACM_LVE_ACT_CNT':'float',
    'MON_12_AGV_LVE_ACT_CNT':'float',
    'CUR_YEAR_PUB_TO_PRV_TRX_PTY_CNT':'float',
    'MON_6_50_UP_ENTR_ACT_CNT':'float',
    'MON_6_50_UP_LVE_ACT_CNT':'float',
    'CUR_YEAR_COUNTER_ENCASH_CNT':'float',
    'MON_12_ACT_OUT_50_UP_CNT_PTY_QTY':'float',
    'MON_12_ACT_IN_50_UP_CNT_PTY_QTY':'float',
    'LAST_12_MON_COR_DPS_TM_PNT_BAL_PEAK_VAL':'float',
    'LAST_12_MON_COR_DPS_DAY_AVG_BAL':'float',
    'CUR_MON_COR_DPS_MON_DAY_AVG_BAL':'float',
    'CUR_YEAR_COR_DMND_DPS_DAY_AVG_BAL':'float',
    'CUR_YEAR_COR_DPS_YEAR_DAY_AVG_INCR':'float',
    'LAST_12_MON_DIF_NM_MON_AVG_TRX_AMT_NAV':'float',
    'LAST_12_MON_MON_AVG_TRX_AMT_NAV':'float',
    'COR_KEY_PROD_HLD_NBR':'float',
    'CUR_YEAR_MID_BUS_INC':'float',
    'AI_STAR_SCO':'float',
    'WTHR_OPN_ONL_ICO':'str',
    'EMP_NBR':'float',
    'REG_CPT':'float',
    'SHH_BCK':'float',
    'HLD_DMS_CCY_ACT_NBR':'float',
    'REG_DT':'float',
    'LGP_HLD_CARD_LVL':'str',
    'OPN_TM':'float',
    'NB_CTC_HLD_IDV_AIO_CARD_SITU':'str',
    'HLD_FGN_CCY_ACT_NBR':'float',
}

removed_features = []
for f in feature_info.keys():
    removed_features.append(count_q_and_nan(df_train,f,ratio=0.5))

removed_features = [i for i in removed_features if i != None]
for rf in removed_features:
    del feature_info[rf]

# 发现有4个类别特征
class_feature_name = []
value_feature_name = []
for feature in feature_info.keys():
    if feature_info[feature] =='float':
        value_feature_name.append(feature)
    else:
        class_feature_name.append(feature)
print(f"数值型特征有{len(value_feature_name)}个， 类别型特征有{len(class_feature_name)}个")

# 数据处理阶段
# 不知道用平均数好还是中位数好，这里先用平均数
def replace_q_with_average(df, feature):
    if '?' not in list(df[feature]):
        # print(f"{feature} do not contain ?")
        return df
    else:  
        values = [i for i in df[feature] if i != '?']
        df[feature].replace("?", sum(values)/len(values), inplace = True)
        
def replace_q_with_G(df, feature):
    if '?' not in list(df[feature]):
        # print(f"{feature} do not contain ?")
        return df
    else:
        df[feature].replace("?", "G", inplace = True)
        
def replace_nan_with_N(df, feature):
    df[feature].replace(np.nan, "N", inplace = True)
    

for feature, kind in feature_info.items():
    if kind =='float':
        replace_q_with_average(df_train, feature = feature)
    else:
        replace_q_with_G(df_train, feature = feature)
        replace_nan_with_N(df_train, feature = feature)

# 将类型特征独热编码

def change_to_onehot(df, feature):
    if feature not in df.columns:
        return df
    df_onehot = pd.get_dummies(df[feature])
    new_columns = [feature+'_'+i for i in df_onehot.columns]
    df_onehot.columns = new_columns
    df_contact = pd.concat([df,df_onehot],axis=1)
    df_contact.drop([feature], axis = 1, inplace = True)
    return df_contact
for f in class_feature_name:
    df_train = change_to_onehot(df_train, f)



x_train, y_train = df_train.drop(['CUST_UID','LABEL'],axis=1), df_train['LABEL']

import lightgbm as lgb
import pandas as pd
from sklearn.model_selection import train_test_split,StratifiedKFold,GridSearchCV,cross_val_score

random_seed = 15

parameters = {
    'max_depth': [2,4,6,8,10,12,14],
    'num_leaves': [ 15, 28, 29, 30, 31,33],
    # 'min_child_samples': [18,19,20,21,22],
    # 'min_child_weight':[0.001,0.002],
    # 'feature_fraction': [0.85, 0.9, 0.95,1.0],
    #  'bagging_fraction': [0.8,0.9,1],
    #  'bagging_freq': [2,3,4],
    #  'reg_alpha' :[0.001,0.01,0.1,1],
    #   'reg_lambda' : [0.001,0.01,0.1,1,10],
}
kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=random_seed)
gbm = lgb.LGBMClassifier(objective = 'binary',
                         is_unbalance = True,
                         metric = 'binary_logloss,auc',
                         learning_rate = 0.1,
                         # num_iterations = 200,   
                        )

gsearch = GridSearchCV(gbm, param_grid=parameters, scoring='roc_auc', cv=kf, n_jobs=-1)
gsearch.fit(x_train, y_train)
print('参数的最佳取值:{0}'.format(gsearch.best_params_))
print('最佳模型得分:{0}'.format(gsearch.best_score_))
# print(gsearch.cv_results_['mean_test_score'])

训练数据的数据大小为：(40000, 51)
测试A榜的数据大小为：(12000, 50)
Remove feature AGN_CNT_RCT_12_MON!
Remove feature AGN_CUR_YEAR_AMT!
Remove feature AGN_CUR_YEAR_WAG_AMT!
Remove feature MON_12_CUST_CNT_PTY_ID!
Remove feature CUR_YEAR_PUB_TO_PRV_TRX_PTY_CNT!
数值型特征有41个， 类别型特征有3个


In [ ]:
df_train.describe()

In [2]:
df_train

NameError: name 'df_train' is not defined